In [6]:
import shutil
import nbimporter
from patch_generator import Patch_Generator

import pandas as pd
import numpy as np
import os
import gc
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
from skimage.io import imread, imshow
from skimage.transform import resize
from keras.models import Model, load_model
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
import time
import tensorflow as tf

# Don't Show Warning Messages
import warnings
warnings.filterwarnings('ignore')
from patchify import patchify, unpatchify
import random
 
%matplotlib inline
import glob
from pathlib import Path
import math

seed = 42
np.random.seed = seed
from keras.utils.np_utils import normalize
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
from keras import backend as K
from numpy import asarray

from keras.utils import normalize
from PIL import Image


# Processing data- Making Patches

In [ ]:
#this function used to crop the unlabeled images along side their pseudo labels and store both mask and image along side the training dataset.
def process_alter(image, label, patches, patch_size, count_,iter_):

    for ptch in range(patches):
        y = np.random.randint(0,IMG_HEIGHT - patch_size+1)
        x = np.random.randint(0,IMG_WIDTH - patch_size+1)
        im_crop = image[ y:y+patch_size, x:x+patch_size]
        lab_crop = label[ y:y+patch_size, x:x+patch_size]
        
        im_crop = cv2.resize(im_crop, (IMG_HEIGHT, IMG_WIDTH))
        lab_crop = cv2.resize(lab_crop, (IMG_HEIGHT, IMG_WIDTH))
#         if (np.mean(lab_crop)!=0):
        
        cv2.imwrite(x_pseudo_pth+str(count_)+'_'+ str(ptch)+ '_'+ str(iter_)+'.png', im_crop)
        cv2.imwrite(y_pseudo_pth+str(count_)+'_'+ str(ptch)+ '_'+ str(iter_)+'.png', lab_crop*255)
        
  

In [ ]:
# only select patches with atleast ones    ### TRAIN DATASET###
data_path_ = r'F:\semi_\Final\unlabeled_images\\*.*'   
save_data_pth = r'F:\semi_\Final\\unlableled_images_patches\\'
# Patch_Generator(data_path_, '', save_data_pth, '', include_large_img=False, with_labels=False)   # generate patches from unlabel dataset

# Training and Sorting

In [ ]:
#self entropy function for measure of uncertanity 
def entropy_error(mask):     
    ent_ = -1 * mask * np.log2(mask+1e-7)
    return  np.sum(ent_)

In [ ]:
def custom_loss(y_true, y_pred):
    # Compute cross entropy loss
    cross_entropy_loss = K.binary_crossentropy(y_true, y_pred)
    
    # Compute IoU loss
    intersection = K.sum(K.abs(y_true * y_pred))
    union = K.sum(y_true) + K.sum(y_pred) - intersection
    iou_loss = 1 - (intersection + 1) / (union + 1)  # Adding 1 to avoid division by zero
    
   
    # Combine the losses
    loss = cross_entropy_loss + 0.1*iou_loss   # Jaccard loss with binary cross entropy
    
    return loss

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)    #0.0001
base_ptch_512.model.compile(optimizer=opt, loss=custom_loss, metrics=['accuracy'])   #focal_loss_with_binary_crossentropy()


In [ ]:
x_unlabeled = []
for img_path in sorted(glob.glob(os.path.join(r'F:\semi_\Final\unlableled_images_patches\*.*'))):

    x_unlabeled.append(img_path)
len(x_unlabeled)

In [ ]:
def sorting_entrop(model, x_unlabeled, mean, std, IMG_HEIGHT):
    print("Calculating Entropies and Sorting")
    entropies_unlab = []
    un_lab_images_fn = []
    for unlab_img_fn in tqdm(x_unlabeled):
        image = cv2.imread(unlab_img_fn,0)
        image = cv2.resize(image, (IMG_HEIGHT, IMG_HEIGHT))/255
     
        image = (image - mean)/ std 
 
        image = image.reshape(1,IMG_HEIGHT,IMG_HEIGHT,1 )
        pred = model.predict(image,verbose=0)
        pred_ = (pred >= 0.5).astype(np.uint8)
        area = np.sum(pred_)/(IMG_HEIGHT*IMG_WIDTH)
        
        
        
        if area > entr_area_th:


            un_lab_images_fn.append(unlab_img_fn)
            entropies_unlab.append(entropy_error(pred))
        del image, pred
    
    print("Sorting")
    combine_list = list(zip(entropies_unlab, un_lab_images_fn))
    sorted_list = sorted(combine_list, key=lambda x:x[0])
    x_unlabeled_sort = [x[1] for x in sorted_list]
    entr_unlabeled_sort = [x[0] for x in sorted_list]
    
    del combine_list, sorted_list, entropies_unlab
    gc.collect()
    
    return x_unlabeled_sort

In [ ]:
checkpointer = tf.keras.callbacks.ModelCheckpoint('20%', verbose=1, save_best_only=True)

callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=5, monitor='val_loss'),
    tf.keras.callbacks.TensorBoard(log_dir='logs')
]
def Self_SS(model, x_unlabeled_sort, mean, std, IMG_HEIGHT, train_data_pth, train_mask_pth, mean_ch = False, main_test=False, btch_sz= 32, iter_ = 0):
    

    X_new_filter = x_unlabeled_sort[: int(len(x_unlabeled_sort)*selected_portion)]   
    
    
    print("Generating Pseudo labels")

    for i, n_img_fn in enumerate(X_new_filter):
        image_org = cv2.imread(n_img_fn,0)
        image = cv2.resize(image_org, (IMG_HEIGHT, IMG_HEIGHT))/255
        image = (image-mean)/std
        image = image.reshape(1,IMG_HEIGHT,IMG_HEIGHT,1 )
        pred = model.predict(image,verbose=0).reshape(IMG_HEIGHT,IMG_HEIGHT)
        pred = cv2.erode(pred.reshape(IMG_HEIGHT,IMG_HEIGHT), kernel, iterations = 1)
        pseudo_lab = np.where(pred>0.5,1,0).astype(np.uint8)

        process_alter(image_org,pseudo_lab , no_of_pathces, altr_ptch_sz, i, iter_)
        
        
    print("Training ...")
    train_list = os.listdir(x_pseudo_pth)
    t_list = train_list[0:int((1-val_set)*len(train_list))]
    v_list = train_list[int((1-val_set)*len(train_list)):]
 
    train_gen = image_data_generator(t_list,x_pseudo_pth,y_pseudo_pth, batch_size=btch_sz, data_aug=True, main_test=main_test, mean_ch =mean_ch )
    val_gen = image_data_generator(v_list,x_pseudo_pth,y_pseudo_pth, batch_size=btch_sz, data_aug=False,main_test=main_test, mean_ch =mean_ch)
    
    
    
    results = model.fit_generator(train_gen, epochs=20, steps_per_epoch=int(len(t_list)/btch_sz),
                                      validation_data=val_gen, validation_steps=int(len(v_list)/btch_sz),
                                      callbacks=callbacks)
    
    
    
    

In [ ]:
def sec_to_(duration):
    hours = duration // 3600
    minutes = (duration - (hours * 3600)) // 60
    seconds = duration - ((hours * 3600) + (minutes * 60))
    msg = f'training elapsed time was {str(hours)} hours, {minutes:4.1f} minutes, {seconds:4.2f} seconds)'
    return msg # print out inferenceduration time

## Self supervised pseudo algorithm 

1. X and Y are input labeled dataset wheras X' is large unlabeled dataset. M is the base model.
2. Inference X' through model M and generate output Y' probalities.
3. Calculate entropy on Y' probalities and sort the X' from low entropy to high entropy. added a minimum area threshold "entr_area_th" for filteration. 
4. Select "selected_portion" from sorted X' and used that to generate pseudo labels. that sample we can call X_pseudo and Y_pseudo.
5. Include these samples with main dataset X and Y and finetune the model M.
6. Exclude the samples X_pseudo from main sorted list. 
7. Repeat from point 4.

In [ ]:
selected_portion = 0.03
iteration_ss = int(1/selected_portion)
iteration_ss = 10
entr_area_th = 0.007
val_set = 0.1

# good_model = 'model_20%_patches.h5'
x_pseudo_pth = r"F:\semi_\Final\x_pseudo\\"
y_pseudo_pth = r"F:\semi_\Final\y_pseudo\\"
kernel = np.ones((5,5), np.uint8)


if os.path.exists(x_pseudo_pth):
    shutil.rmtree(x_pseudo_pth)
    shutil.rmtree(y_pseudo_pth)
print("Copying folder")
shutil.copytree(train_data_pth, x_pseudo_pth)
shutil.copytree(train_mask_pth, y_pseudo_pth)


In [ ]:
# x_unlab_sort__main = sorting_entrop(base_ptch_512.model,x_unlabeled, mean, std, IMG_HEIGHT)
# print(len(x_unlab_sort__main))
# with open('unlab_list_bi.txt', 'wb') as file:
#     # Dump the list into the file
#     pickle.dump(x_unlab_sort__main, file)

In [ ]:
with open('unlab_list_bi.txt', 'rb') as file:
    # Load the list from the file
    x_unlab_sort__main = pickle.load(file)

In [ ]:
len(x_unlab_sort__main)

In [ ]:

for iter_ in range(iteration_ss):
    
    print(f"------Self Supervised Epoch: {iter_}/{iteration_ss}-----")
    
    

    now = time.time()
    

    no_of_pathces = 2
    altr_ptch_sz = 356
    IMG_HEIGHT = 512
    IMG_WIDTH=512
    Self_SS(base_ptch_512.model, x_unlab_sort__main, mean, std, IMG_HEIGHT, train_data_pth, train_mask_pth, mean_ch = False, btch_sz=32, iter_=iter_ )
    
    
    
    
    x_unlab_sort__main = x_unlab_sort__main[int(len(x_unlab_sort__main)*selected_portion):] 
    print(len(x_unlab_sort__main))
    end_r = time.time()
    inf_t = int(end_r - now)
    print(sec_to_(inf_t))
    print("Testing")
    
    Large_image_metric(base_ptch_512.model, test_data_list, data_path, lab_path)


#     if (jaccard_sim > prev_sim):
#     selected_portion += selected_portion

    good_model = f'model_100%_ss_iter{str(iter_)}.h5'
    base_ptch_512.model.save_weights(good_model)
#     prev_sim = jaccard_sim
#     else:

#         base_ptch_512.load_weights(good_model)
#         print(f"Loaded model with this sim::{prev_sim}")



## Inference

In [ ]:
base_ptch_512.model.load_weights('model_100%_ss_iter5.h5') 

In [ ]:

kernel_ = np.ones((10,10), np.uint8)
def inf_onlyImage(model,data_path, img_pth):
    image = cv2.imread(data_path + img_pth, 0)
    init_shape = image.shape
    aspect_ratio = image.shape[1] / image.shape[0]
    
    new_size = 2400
    if aspect_ratio > 1:
      
        new_width = new_size
        new_height = int(new_width / aspect_ratio)
    else:
      
        new_height = new_size
        new_width = int(new_height * aspect_ratio)
    image = cv2.resize(image, (new_width, new_height))

    
    # pad image to make sure its size is a multiple of the patch size
    padded_image = pad_image(image, IMG_HEIGHT)
  
  
    original_shape = image.shape

  
    
    patches = patchify(padded_image, (IMG_HEIGHT, IMG_WIDTH), step=step_size)
  
   
    predicted_patches = []

    for i in range(patches.shape[0]):
        for j in range(patches.shape[1]):
            single_patch = patches[i,j,:,:] / 255.0
            single_patch = (single_patch - mean) / std
            single_patch_input = single_patch.reshape(1, IMG_WIDTH, IMG_HEIGHT, 1)
            single_patch_prediction = (model.predict(single_patch_input, verbose=0))
            predicted_patches.append(single_patch_prediction)
          
    predicted_patches = np.array(predicted_patches)
    predicted_patches_reshaped = np.reshape(predicted_patches, (patches.shape[0], patches.shape[1], IMG_HEIGHT, IMG_WIDTH))
    
    reconstructed_image = unpatchify(predicted_patches_reshaped, (padded_image.shape[0], padded_image.shape[1]))

    
    # remove padding
    reconstructed_image = reconstructed_image[:original_shape[0], :original_shape[1]]

    reconstructed_image = (reconstructed_image > 0.5).astype(np.uint8)
    
    
    image = cv2.resize(image,(init_shape[1],init_shape[0]) )
    reconstructed_image = cv2.resize(reconstructed_image, (init_shape[1],init_shape[0]))
    
    return reconstructed_image, init_shape[1]*init_shape[0]
    


In [ ]:
data_pth  = r'E:\\MuathStudy\\New folder (8)\\' 
save_data_pth   = r'E:\\MuathStudy\\New folder (7)\\' 
img_data_list = os.listdir(data_pth)


In [ ]:
file_path = r'E:\\MuathStudy\\area.txt'
file_path_1 = r'E:\\MuathStudy\\count.txt'
file = open(file_path, 'w')
file_1 = open(file_path_1, 'w')

for fn in img_data_list:
    preds_test_thresh , h_w= inf_onlyImage(base_ptch_512.model,data_pth, fn)
    
#     count = np.count_nonzero(preds_test_thresh == 1) this is for COX not importent here
#     area = count/h_w
    cv2.imwrite(save_data_pth +'M'+ fn, preds_test_thresh*255)
#     file.write(fn+ f" : {area}")
#     file_1.write(fn+ f" : {count}")
#     print(fn)

    
file.close()
file_1.close()